In [23]:
import sys
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory

from matplotlib import pyplot
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd
import seaborn as sns

import json

# vgg16 model used for transfer learning on the dogs and cats dataset
from keras.applications.vgg16 import VGG16
from keras.models import Model

#physical_devices = tf.config.list_physical_devices('GPU') 
#tf.config.experimental.set_memory_growth(physical_devices[0], True)
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [24]:
BATCH_SIZE = 32
EPOCHS = 200
IMG_SIZE = 224
LABELS = ["female", "male"]
initial_epochs = 100
fine_tune_epochs = 100
# Fine-tune from this layer onwards
fine_tune_at = 100

# Load the datasets
# In this POC, perturbed data refers to facial data processed with makeup
preprocessing_fp = "preprocessing"

In [25]:
# define cnn model

def define_model():
    
	model = Sequential()
    
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model

In [26]:
# Transfer Learning
# define cnn model
def define_model():
	# load model
	model = VGG16(include_top=False, input_shape=(224, 224, 3))
	# mark loaded layers as not trainable
	for layer in model.layers:
		layer.trainable = False
	# add new classifier layers
	flat1 = Flatten()(model.layers[-1].output)
	class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
	output = Dense(1, activation='sigmoid')(class1)
	# define new model
	model = Model(inputs=model.inputs, outputs=output)
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model


# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()
 
# run the test harness for evaluating a model
def run_test_harness():
	# define model
    model = define_model()
    
	# create data generators
	#train_datagen = ImageDataGenerator(rescale=1.0/255.0,
	#	width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
	#test_datagen = ImageDataGenerator(rescale=1.0/255.0)
    
    # 1
    test_datagen = ImageDataGenerator(rescale=1/255)
    
    test_it = test_datagen.flow_from_directory(
        os.path.join(preprocessing_fp, "val_data"),
        target_size=(224, 224),
        batch_size=BATCH_SIZE,
        class_mode="categorical",
    )
    
    # 2
    #class_subset = sorted(os.listdir(os.path.join(preprocessing_fp, "val_data")))[:10]
    test_generator = ImageDataGenerator(rescale=1/255)
    testgen = test_generator.flow_from_directory(
        os.path.join(preprocessing_fp, "val_data"),
        target_size=(128, 128),
        batch_size=1,
        class_mode=None,
        classes=class_subset, 
        shuffle=False,
    )

	# prepare iterators
	#train_it = train_datagen.flow_from_directory('dataset_dogs_vs_cats/train/',
	#	class_mode='binary', batch_size=64, target_size=(200, 200))
	#test_it = test_datagen.flow_from_directory('dataset_dogs_vs_cats/test/',
	#	class_mode='binary', batch_size=64, target_size=(200, 200))
    
	# fit model
    history = model.fit_generator(
        train_it, 
        steps_per_epoch=len(train_it),
		validation_data=test_it, 
        validation_steps=len(test_it), 
        epochs=50, 
        verbose=0
    )
    
	# evaluate model
    _, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)
    print('> %.3f' % (acc * 100.0))

    # learning curves
    summarize_diagnostics(history)
 
# entry point, run the test harness
run_test_harness()

InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory

In [27]:
# Transfer Learning
# define cnn model
def define_model():
	# load model
	model = VGG16(include_top=False, input_shape=(224, 224, 3))
	# mark loaded layers as not trainable
	for layer in model.layers:
		layer.trainable = False
	# add new classifier layers
	flat1 = Flatten()(model.layers[-1].output)
	class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
	output = Dense(1, activation='sigmoid')(class1)
	# define new model
	model = Model(inputs=model.inputs, outputs=output)
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model